### Palm API connection and checking llm

In [27]:
from langchain.llms import GooglePalm
from key_params import palmapi_key

api_key = palmapi_key

llm = GooglePalm(google_api_key=api_key,temperature=0.1)
poem = llm("write a poem about a reindeer named tony tony chopper")
print(poem)

, the reindeer,
was born on the northern tundra.
he had a red nose and a long tail,
and he loved to play in the snow.

one day, tony chopper was playing
when he saw a group of children sledding.
he wanted to join in,
but he was too shy.

the children saw tony chopper standing by himself,
and they invited him to play.
tony chopper was so happy,
and he had a great time sledding with the children.

from that day on, tony chopper was no longer shy.
he loved to play with the children,
and he always made sure to help them have a good time.

tony chopper was a kind and gentle reindeer,
and he was loved by everyone who knew him.
he was a true friend,
and he will always be remembered.


### Connect to SQL database and try out basic commands

In [28]:
from langchain.utilities import SQLDatabase

db_user = "root"
db_password = "sqlpasswd"
db_host = "localhost"
db_name = "vehicle_insurance"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
print(db.table_info)


CREATE TABLE team10_application (
	`Application_Id` VARCHAR(20) NOT NULL, 
	`Vehicle_Id` VARCHAR(20) NOT NULL, 
	`Application_Status` CHAR(8) NOT NULL, 
	`Coverage` VARCHAR(50) NOT NULL, 
	`Cust_Id` INTEGER NOT NULL, 
	PRIMARY KEY (`Application_Id`, `Cust_Id`), 
	CONSTRAINT `R_93` FOREIGN KEY(`Cust_Id`) REFERENCES team10_customer (`Cust_Id`) ON DELETE CASCADE ON UPDATE CASCADE
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from team10_application table:
Application_Id	Vehicle_Id	Application_Status	Coverage	Cust_Id
AP20153	557002	RECEIVED	Liability	153
AP20176	557005	ACCEPTED	Collision	155
AP20182	557001	ACCEPTED	Collision	152
*/


CREATE TABLE team10_claim (
	`Claim_Id` INTEGER NOT NULL, 
	`Agreement_Id` VARCHAR(20) NOT NULL, 
	`Application_Id` VARCHAR(20) NOT NULL, 
	`Claim_Amount` INTEGER NOT NULL, 
	`Incident_Id` INTEGER NOT NULL, 
	`Damage_Type` VARCHAR(20) NOT NULL, 
	`Date_Of_Claim` DATE NOT NULL, 
	`Claim_Status` CHAR(10) NOT NULL, 
	`Cust_Id` INTE

In [29]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db,verbose=True)
q1 = db_chain.run("How many unique vehicles do we have?")



> Entering new SQLDatabaseChain chain...
How many unique vehicles do we have?
SQLQuery:SELECT count(DISTINCT Vehicle_Id) FROM team10_vehicle
SQLResult: [(17,)]
Answer:17
> Finished chain.


That's correct!

In [30]:
q2 = db_chain.run("How many SUV vehicles do we have?")



> Entering new SQLDatabaseChain chain...
How many SUV vehicles do we have?
SQLQuery:SELECT COUNT(*) FROM team10_vehicle WHERE Vehicle_Type = 'SUV'
SQLResult: [(8,)]
Answer:1
> Finished chain.


That's right as well!

In [31]:
q3 = db_chain.run("How many 5 seaters do we have below 2000000?")



> Entering new SQLDatabaseChain chain...
How many 5 seaters do we have below 2000000?
SQLQuery:SELECT count(*) FROM team10_vehicle WHERE Vehicle_Number_Of_Seat = 5 AND Vehicle_Value < 2000000
SQLResult: [(4,)]
Answer:4
> Finished chain.


In [32]:
q4 = db_chain.run("How many Sedans do we have registered in Karnataka?")



> Entering new SQLDatabaseChain chain...
How many Sedans do we have registered in Karnataka?
SQLQuery:SELECT COUNT(*) FROM team10_vehicle WHERE Vehicle_Type = "Sedan" AND Vehicle_Registration_Number LIKE "KA%"
SQLResult: [(3,)]
Answer:3
> Finished chain.


Wrong! Now the model had to understand that the vehicle's registration number starting with KA would mean that its registered Karnataka, and had to use that.

In [45]:
q5 = db_chain.run("How many of the staff are unmarried females?")



> Entering new SQLDatabaseChain chain...
How many of the staff are unmarried females?
SQLQuery:SELECT COUNT(*) FROM team10_staff WHERE Staff_Gender = 'F' AND Staff_Marital_Status = 'Single'
SQLResult: [(1,)]
Answer:1
> Finished chain.


In [38]:
q6 = db_chain.run("Retrieve Customer details whose Claim Amount is less than Coverage Amount and Claim Amount is greater than Sum of (CLAIM_SETTLEMENT_ID, VEHICLE_ID, CLAIM_ID, CUST_ID)")



> Entering new SQLDatabaseChain chain...
Retrieve Customer details whose Claim Amount is less than Coverage Amount and Claim Amount is greater than Sum of (CLAIM_SETTLEMENT_ID, VEHICLE_ID, CLAIM_ID, CUST_ID)
SQLQuery:SELECT t10_customer.Cust_Id, t10_customer.Cust_FName, t10_customer.Cust_LName FROM t10_customer JOIN t10_insurance_policy ON t10_customer.Cust_Id = t10_insurance_policy.Cust_Id JOIN t10_insurance_policy_coverage ON t10_insurance_policy.Agreement_id = t10_insurance_policy_coverage.Agreement_id AND t10_insurance_policy.Application_Id = t10_insurance_policy_coverage.Application_Id AND t10_insurance_policy.Cust_Id = t10_insurance_policy_coverage.Cust_Id WHERE t10_insurance_policy_coverage.Coverage_Amount > t10_insurance_policy.Claim_Amount AND t10_insurance_policy.Claim_Amount > (SELECT SUM(t10_premium_payment_receipt.Premium_Payment_Amount) FROM t10_premium_payment_receipt WHERE t10_premium_payment_receipt.Cust_Id = t10_customer.Cust_Id)

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'vehicle_insurance.t10_customer' doesn't exist")
[SQL: SELECT t10_customer.Cust_Id, t10_customer.Cust_FName, t10_customer.Cust_LName FROM t10_customer JOIN t10_insurance_policy ON t10_customer.Cust_Id = t10_insurance_policy.Cust_Id JOIN t10_insurance_policy_coverage ON t10_insurance_policy.Agreement_id = t10_insurance_policy_coverage.Agreement_id AND t10_insurance_policy.Application_Id = t10_insurance_policy_coverage.Application_Id AND t10_insurance_policy.Cust_Id = t10_insurance_policy_coverage.Cust_Id WHERE t10_insurance_policy_coverage.Coverage_Amount > t10_insurance_policy.Claim_Amount AND t10_insurance_policy.Claim_Amount > (SELECT SUM(t10_premium_payment_receipt.Premium_Payment_Amount) FROM t10_premium_payment_receipt WHERE t10_premium_payment_receipt.Cust_Id = t10_customer.Cust_Id)]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [42]:
q6 = db_chain.run("SQLQuery is just this nothing else SELECT DISTINCT cus.* FROM team10_claim c INNER JOIN team10_customer cus ON c.Cust_Id = cus.Cust_Id INNER JOIN team10_claim_settlement cs ON cus.Cust_Id = cs.Cust_Id INNER JOIN team10_coverage cov ON cs.Coverage_Id = cov.Coverage_Id WHERE c.claim_amount>(cs.claim_settlement_id+cs.vehicle_id+cs.claim_id+cs.cust_id) AND c.claim_amount<cov.coverage_amount")



> Entering new SQLDatabaseChain chain...
SQLQuery is just this nothing else SELECT DISTINCT cus.* FROM team10_claim c INNER JOIN team10_customer cus ON c.Cust_Id = cus.Cust_Id INNER JOIN team10_claim_settlement cs ON cus.Cust_Id = cs.Cust_Id INNER JOIN team10_coverage cov ON cs.Coverage_Id = cov.Coverage_Id WHERE c.claim_amount>(cs.claim_settlement_id+cs.vehicle_id+cs.claim_id+cs.cust_id) AND c.claim_amount<cov.coverage_amount
SQLQuery:SELECT DISTINCT cus.* FROM team10_claim c INNER JOIN team10_customer cus ON c.Cust_Id = cus.Cust_Id INNER JOIN team10_claim_settlement cs ON cus.Cust_Id = cs.Cust_Id INNER JOIN team10_coverage cov ON cs.Coverage_Id = cov.Coverage_Id WHERE c.claim_amount>(cs.claim_settlement_id+cs.vehicle_id+cs.claim_id+cs.cust_id) AND c.claim_amount<cov.coverage_amount
SQLResult: [(152, 'Warden', 'Tomcic', datetime.date(1994, 10, 29), 'M', '21, Rostov St., Russia', 7648683324, 'warden@gmail.com', '83742937', 'Single', 982349634), (153, 'Blathnat', 'Otto', datetime.date

In [44]:
q7 = db_chain.run("Select Customers who have more than one Vehicle, where the premium for one of the Vehicles is not paid and it is involved in accident")



> Entering new SQLDatabaseChain chain...
Select Customers who have more than one Vehicle, where the premium for one of the Vehicles is not paid and it is involved in accident
SQLQuery:SELECT t10_customer.Cust_Id, t10_customer.Cust_FName, t10_customer.Cust_LName, t10_vehicle.Vehicle_Id
FROM t10_customer
JOIN t10_vehicle ON t10_customer.Cust_Id = t10_vehicle.Cust_Id
JOIN t10_incident_report ON t10_vehicle.Vehicle_Id = t10_incident_report.Vehicle_Id
JOIN t10_premium_payment_receipt ON t10_incident_report.Vehicle_Id = t10_premium_payment_receipt.Vehicle_Id
WHERE t10_premium_payment_receipt.Premium_Payment_Id IS NULL AND t10_vehicle.Vehicle_Id IN (
SELECT t10_vehicle.Vehicle_Id
FROM t10_vehicle
GROUP BY t10_vehicle.Cust_Id
HAVING COUNT(t10_vehicle.Vehicle_Id) > 1
)

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'vehicle_insurance.t10_customer' doesn't exist")
[SQL: SELECT t10_customer.Cust_Id, t10_customer.Cust_FName, t10_customer.Cust_LName, t10_vehicle.Vehicle_Id
FROM t10_customer
JOIN t10_vehicle ON t10_customer.Cust_Id = t10_vehicle.Cust_Id
JOIN t10_incident_report ON t10_vehicle.Vehicle_Id = t10_incident_report.Vehicle_Id
JOIN t10_premium_payment_receipt ON t10_incident_report.Vehicle_Id = t10_premium_payment_receipt.Vehicle_Id
WHERE t10_premium_payment_receipt.Premium_Payment_Id IS NULL AND t10_vehicle.Vehicle_Id IN (
SELECT t10_vehicle.Vehicle_Id
FROM t10_vehicle
GROUP BY t10_vehicle.Cust_Id
HAVING COUNT(t10_vehicle.Vehicle_Id) > 1
)]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [51]:
q7 = db_chain.run("SQLQuery is the line, and not its result: SELECT DISTINCT (Vehicle_Id) FROM TEAM10_VEHICLE AS T1 INNER JOIN TEAM10_CUSTOMER AS T2 ON T1.Cust_Id = T2.Cust_Id INNER JOIN TEAM10_PREMIUM_PAYMENT_RECEIPT AS T3 ON T2.Cust_Id = T3.Cust_Id INNER JOIN TEAM10_PREMIUM_PAYMENT AS T4 ON T3.Premium_Payment_ID = T4.Premium_Payment_Id WHERE T1.Vehicle_Number < T4.Premium_Payment_Amount AND Policy_Number = Policy_Id;")



> Entering new SQLDatabaseChain chain...
SQLQuery is the line, and not its result: SELECT DISTINCT (Vehicle_Id) FROM TEAM10_VEHICLE AS T1 INNER JOIN TEAM10_CUSTOMER AS T2 ON T1.Cust_Id = T2.Cust_Id INNER JOIN TEAM10_PREMIUM_PAYMENT_RECEIPT AS T3 ON T2.Cust_Id = T3.Cust_Id INNER JOIN TEAM10_PREMIUM_PAYMENT AS T4 ON T3.Premium_Payment_ID = T4.Premium_Payment_Id WHERE T1.Vehicle_Number < T4.Premium_Payment_Amount AND Policy_Number = Policy_Id;
SQLQuery:SELECT DISTINCT (Vehicle_Id) FROM TEAM10_VEHICLE AS T1 INNER JOIN TEAM10_CUSTOMER AS T2 ON T1.Cust_Id = T2.Cust_Id INNER JOIN TEAM10_PREMIUM_PAYMENT_RECEIPT AS T3 ON T2.Cust_Id = T3.Cust_Id INNER JOIN TEAM10_PREMIUM_PAYMENT AS T4 ON T3.Premium_Payment_ID = T4.Premium_Payment_Id WHERE T1.Vehicle_Number < T4.Premium_Payment_Amount AND Policy_Number = Policy_Id;
SQLResult: [(557001,), (557014,), (557016,), (557007,)]
Answer:SELECT DISTINCT (Vehicle_Id) FROM TEAM10_VEHICLE AS T1 INNER JOIN TEAM10_CUSTOMER AS T2 ON T1.Cust_Id = T2.Cust_Id INNE

In [52]:
q7 = db_chain.run("Retrieve Customer and Vehicle details who has been involved in an incident and claim status is pending")



> Entering new SQLDatabaseChain chain...
Retrieve Customer and Vehicle details who has been involved in an incident and claim status is pending
SQLQuery:SELECT t1.Cust_Id, t1.Vehicle_Id, t1.Vehicle_Registration_Number, t1.Vehicle_Type, t1.Vehicle_Size, t1.Vehicle_Number_Of_Seat, t1.Vehicle_Manufacturer, t1.Vehicle_Engine_Number, t1.Vehicle_Chasis_Number, t1.Vehicle_Number, t1.Vehicle_Model_Number
FROM team10_vehicle AS t1
INNER JOIN team10_incident_report AS t2 ON t1.Vehicle_Id = t2.Vehicle_Id
INNER JOIN team10_customer AS t3 ON t1.Cust_Id = t3.Cust_Id
WHERE t2.Claim_Status = 'Pending'

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 't2.Claim_Status' in 'where clause'")
[SQL: SELECT t1.Cust_Id, t1.Vehicle_Id, t1.Vehicle_Registration_Number, t1.Vehicle_Type, t1.Vehicle_Size, t1.Vehicle_Number_Of_Seat, t1.Vehicle_Manufacturer, t1.Vehicle_Engine_Number, t1.Vehicle_Chasis_Number, t1.Vehicle_Number, t1.Vehicle_Model_Number
FROM team10_vehicle AS t1
INNER JOIN team10_incident_report AS t2 ON t1.Vehicle_Id = t2.Vehicle_Id
INNER JOIN team10_customer AS t3 ON t1.Cust_Id = t3.Cust_Id
WHERE t2.Claim_Status = 'Pending']
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [53]:
q7 = db_chain.run("SQLQuery is the line: SELECT * FROM TEAM10_VEHICLE NATURAL JOIN TEAM10_CUSTOMER WHERE Policy_Id IN (SELECT Policy_Number FROM TEAM10_INSURANCE_POLICY WHERE Agreement_ID IN (SELECT Agreement_ID FROM TEAM10_CLAIM WHERE claim_status = 'PENDING'))")



> Entering new SQLDatabaseChain chain...
SQLQuery is the line: SELECT * FROM TEAM10_VEHICLE NATURAL JOIN TEAM10_CUSTOMER WHERE Policy_Id IN (SELECT Policy_Number FROM TEAM10_INSURANCE_POLICY WHERE Agreement_ID IN (SELECT Agreement_ID FROM TEAM10_CLAIM WHERE claim_status = 'PENDING'))
SQLQuery:SELECT * FROM team10_vehicle NATURAL JOIN team10_customer WHERE Policy_Id IN (SELECT Policy_Number FROM team10_insurance_policy WHERE Agreement_ID IN (SELECT Agreement_ID FROM team10_claim WHERE claim_status = 'PENDING'))
SQLResult: [(156, 557008, '2022156/100/2', None, 'AP12HS1234', 2250000, 'Sedan', 1500, 4, 'Honda', 132642372, 139103882, '1487', 'M18395HG216', 'Ramond', 'Sanchez', datetime.date(1993, 12, 21), 'M', '10, Downing Street, England', 9169428111, 'ramsan@gmail.com', '2174311682', 'Married', 39164821), (158, 557012, '2021158/100/2', 'DSA06735', 'MH34HJ3401', 1200000, 'SUV', 3000, 7, 'Mahindra', 843535178, 312344582, '1950', 'M4378XH211', 'Kiara', 'Advani', datetime.date(1999, 10, 1),

### Few shot learning

In [56]:
few_shots = [
    {'Question' : "How many unique vehicles do we have?",
     'SQLQuery' : "SELECT count(DISTINCT Vehicle_Id) FROM team10_vehicle",
     'SQLResult': "[(17,)]",
     'Answer' : q1},
    {'Question': "How many 5 seaters do we have below 2000000?",
     'SQLQuery':"SELECT count(*) FROM team10_vehicle WHERE Vehicle_Number_Of_Seat = 5 AND Vehicle_Value < 2000000",
     'SQLResult': "[(4,)]",
     'Answer': q3},
    {'Question' : "Retrieve Customer details whose Claim Amount is less than Coverage Amount and Claim Amount is greater than Sum of (CLAIM_SETTLEMENT_ID, VEHICLE_ID, CLAIM_ID, CUST_ID)" ,
      'SQLQuery': "SELECT DISTINCT cus.* FROM team10_claim c INNER JOIN team10_customer cus ON c.Cust_Id = cus.Cust_Id INNER JOIN team10_claim_settlement cs ON cus.Cust_Id = cs.Cust_Id INNER JOIN team10_coverage cov ON cs.Coverage_Id = cov.Coverage_Id WHERE c.claim_amount>(cs.claim_settlement_id+cs.vehicle_id+cs.claim_id+cs.cust_id) AND c.claim_amount<cov.coverage_amount",
      'SQLResult': "[(152, 'Warden', 'Tomcic', datetime.date(1994, 10, 29), 'M', '21, Rostov St., Russia', 7648683324, 'warden@gmail.com', '83742937', 'Single', 982349634), (153, 'Blathnat', 'Otto', datetime.date(1982, 11, 12), 'F', '9549, Prospect Lane, Livonia', 7628483632, 'ottotto@gmail.com', '12389237', 'Divorced', 98345628), (155, 'Nuria', 'Sigurdsson', datetime.date(1969, 4, 27), 'F', '180, West Court, Westland', 9135566527, 'nuriarox@hotmail.com', '8734234', 'Single', 982374897), (159, 'Homer', 'Kumar', datetime.date(1984, 6, 15), 'M', '6897, Rose Quartz Street', 9879863679, 'homerk@gmail.com', '9472364739', 'Divorced', 82977857), (157, 'Javier', 'Hernandez', datetime.date(1989, 5, 12), 'M', '12, Central Avenue, LA', 9285428612, 'jhernan@yahoo.com', '2271512511', 'Single', 31572420)]",
      'Answer' : "[(152, 'Warden', 'Tomcic', datetime.date(1994, 10, 29), 'M', '21, Rostov St., Russia', 7648683324, 'warden@gmail.com', '83742937', 'Single', 982349634), (153, 'Blathnat', 'Otto', datetime.date(1982, 11, 12), 'F', '9549, Prospect Lane, Livonia', 7628483632, 'ottotto@gmail.com', '12389237', 'Divorced', 98345628), (155, 'Nuria', 'Sigurdsson', datetime.date(1969, 4, 27), 'F', '180, West Court, Westland', 9135566527, 'nuriarox@hotmail.com', '8734234', 'Single', 982374897), (159, 'Homer', 'Kumar', datetime.date(1984, 6, 15), 'M', '6897, Rose Quartz Street', 9879863679, 'homerk@gmail.com', '9472364739', 'Divorced', 82977857), (157, 'Javier', 'Hernandez', datetime.date(1989, 5, 12), 'M', '12, Central Avenue, LA', 9285428612, 'jhernan@yahoo.com', '2271512511', 'Single', 31572420)]"},
    {'Question': "Select Customers who have more than one Vehicle, where the premium for one of the Vehicles is not paid and it is involved in accident",
     'SQLQuery' : "SELECT DISTINCT (Vehicle_Id) FROM TEAM10_VEHICLE AS T1 INNER JOIN TEAM10_CUSTOMER AS T2 ON T1.Cust_Id = T2.Cust_Id INNER JOIN TEAM10_PREMIUM_PAYMENT_RECEIPT AS T3 ON T2.Cust_Id = T3.Cust_Id INNER JOIN TEAM10_PREMIUM_PAYMENT AS T4 ON T3.Premium_Payment_ID = T4.Premium_Payment_Id WHERE T1.Vehicle_Number < T4.Premium_Payment_Amount AND Policy_Number = Policy_Id",
     'SQLResult': "[(557001,), (557014,), (557016,), (557007,)]",
     'Answer' : "[(557001,), (557014,), (557016,), (557007,)]"
     },
     {'Question': "Retrieve Customer and Vehicle details who has been involved in an incident and claim status is pending",
     'SQLQuery' : "SELECT * FROM TEAM10_VEHICLE NATURAL JOIN TEAM10_CUSTOMER WHERE Policy_Id IN (SELECT Policy_Number FROM TEAM10_INSURANCE_POLICY WHERE Agreement_ID IN (SELECT Agreement_ID FROM TEAM10_CLAIM WHERE claim_status = 'PENDING'))",
     'SQLResult': "[(156, 557008, '2022156/100/2', None, 'AP12HS1234', 2250000, 'Sedan', 1500, 4, 'Honda', 132642372, 139103882, '1487', 'M18395HG216', 'Ramond', 'Sanchez', datetime.date(1993, 12, 21), 'M', '10, Downing Street, England', 9169428111, 'ramsan@gmail.com', '2174311682', 'Married', 39164821), (158, 557012, '2021158/100/2', 'DSA06735', 'MH34HJ3401', 1200000, 'SUV', 3000, 7, 'Mahindra', 843535178, 312344582, '1950', 'M4378XH211', 'Kiara', 'Advani', datetime.date(1999, 10, 1), 'F', '561, Church Street, Hyderabad', 9776833668, 'kiaraad@gmail.com', '29723614893', 'Married', 65637863), (159, 557014, '2021159/100/1', 'DSB06459', 'AS11GF3872', 1900000, 'SUV', 1000, 5, 'Mahindra', 898765178, 374684582, '1160', 'M8075XH211', 'Homer', 'Kumar', datetime.date(1984, 6, 15), 'M', '6897, Rose Quartz Street', 9879863679, 'homerk@gmail.com', '9472364739', 'Divorced', 82977857), (160, 557015, '2020160/100/1', 'DSB06456', 'LA03LM9183', 1700000, 'SUV', 4000, 7, 'Toyota', 123455178, 365478582, '1690', 'M7568XH211', 'Sara', 'Ujjwal', datetime.date(1998, 2, 18), 'F', '283, MG Road, Delhi', 8798637829, 'ujjwalsara@gmail.com', '5843643572', 'Married', 32142862), (155, 557005, '2021155/100/1', 'DSC08539', 'TS34RG1873', 200000, 'Mini', 800, 4, 'Maruti', 825564417, 322533223, '1655', 'M38725TH183', 'Nuria', 'Sigurdsson', datetime.date(1969, 4, 27), 'F', '180, West Court, Westland', 9135566527, 'nuriarox@hotmail.com', '8734234', 'Single', 982374897)]",
     'Answer' : "[(156, 557008, '2022156/100/2', None, 'AP12HS1234', 2250000, 'Sedan', 1500, 4, 'Honda', 132642372, 139103882, '1487', 'M18395HG216', 'Ramond', 'Sanchez', datetime.date(1993, 12, 21), 'M', '10, Downing Street, England', 9169428111, 'ramsan@gmail.com', '2174311682', 'Married', 39164821), (158, 557012, '2021158/100/2', 'DSA06735', 'MH34HJ3401', 1200000, 'SUV', 3000, 7, 'Mahindra', 843535178, 312344582, '1950', 'M4378XH211', 'Kiara', 'Advani', datetime.date(1999, 10, 1), 'F', '561, Church Street, Hyderabad', 9776833668, 'kiaraad@gmail.com', '29723614893', 'Married', 65637863), (159, 557014, '2021159/100/1', 'DSB06459', 'AS11GF3872', 1900000, 'SUV', 1000, 5, 'Mahindra', 898765178, 374684582, '1160', 'M8075XH211', 'Homer', 'Kumar', datetime.date(1984, 6, 15), 'M', '6897, Rose Quartz Street', 9879863679, 'homerk@gmail.com', '9472364739', 'Divorced', 82977857), (160, 557015, '2020160/100/1', 'DSB06456', 'LA03LM9183', 1700000, 'SUV', 4000, 7, 'Toyota', 123455178, 365478582, '1690', 'M7568XH211', 'Sara', 'Ujjwal', datetime.date(1998, 2, 18), 'F', '283, MG Road, Delhi', 8798637829, 'ujjwalsara@gmail.com', '5843643572', 'Married', 32142862), (155, 557005, '2021155/100/1', 'DSC08539', 'TS34RG1873', 200000, 'Mini', 800, 4, 'Maruti', 825564417, 322533223, '1655', 'M38725TH183', 'Nuria', 'Sigurdsson', datetime.date(1969, 4, 27), 'F', '180, West Court, Westland', 9135566527, 'nuriarox@hotmail.com', '8734234', 'Single', 982374897)]"
     },
     {'Question': "How many Sedans do we have registered in Karnataka?" ,
     'SQLQuery' : "SELECT COUNT(*) FROM team10_vehicle WHERE Vehicle_Type = 'Sedan' AND Vehicle_Registration_Number LIKE 'KA%'",
     'SQLResult': "[(3,)]",
     'Answer': "3",
     }
]

Vectorize these prompts: make embeddings

In [55]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<?, ?B/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<?, ?B/s] 
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 10.6MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 120kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 1.36MB/s]
pytorch_model.bin: 100%|██████████| 438M/438M [02:05<00:00, 3.48MB/s] 
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 51.1kB/s]
special_tokens_map.json: 100%|██████████| 239/239 [00:00<?, ?B/s] 
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.67MB/s]
tokenizer_config.json: 100%|██████████| 363/363 [00:00<?, ?B/s] 
train_script.py: 100%|██████████| 13.1k/13.1k [00:00<?, ?B/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 8.56MB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 350kB/s]


NameError: name 'few_shots' is not defined

In [57]:
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [69]:
embeddings.embed_query("Select all vehicles which have premium more than its vehicle number.")

[-0.03987126052379608,
 0.02790747955441475,
 -0.034822408109903336,
 0.05535691976547241,
 -0.005538855213671923,
 -0.03472528234124184,
 -0.046069804579019547,
 0.08738318830728531,
 -0.033262740820646286,
 -0.009508905000984669,
 -0.002892441116273403,
 0.07769192010164261,
 -0.012292272411286831,
 0.008031090721487999,
 -0.0031386278569698334,
 -0.017178967595100403,
 0.015819046646356583,
 0.008790720254182816,
 0.07144501805305481,
 -0.014008678495883942,
 -0.036808740347623825,
 0.03363267332315445,
 -0.007894644513726234,
 0.0026233450043946505,
 0.002033426659181714,
 0.05276213213801384,
 0.07828836888074875,
 -0.030288536101579666,
 0.02304033562541008,
 -0.03020642325282097,
 0.01899624615907669,
 0.02582056261599064,
 0.06344694644212723,
 -0.03947734832763672,
 1.4513030919260927e-06,
 -0.005599193274974823,
 -0.06104004755616188,
 -0.007256170269101858,
 -0.004094813484698534,
 0.037696901708841324,
 -0.001782639417797327,
 0.1001347005367279,
 0.0019179126247763634,
 -0

and store it in a Chroma vector search db

In [60]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

Now we need to instruct the LLM to generate two similar prompts to the user given prompt, and use the results from those to get our answer.

In [87]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "Select Customers who have more than one Vehicle, where the premium for one of the Vehicles is not paid and it is involved in accident"})

[{'Answer': '[(557001,), (557014,), (557016,), (557007,)]',
  'Question': 'Select Customers who have more than one Vehicle, where the premium for one of the Vehicles is not paid and it is involved in accident',
  'SQLQuery': 'SELECT DISTINCT (Vehicle_Id) FROM TEAM10_VEHICLE AS T1 INNER JOIN TEAM10_CUSTOMER AS T2 ON T1.Cust_Id = T2.Cust_Id INNER JOIN TEAM10_PREMIUM_PAYMENT_RECEIPT AS T3 ON T2.Cust_Id = T3.Cust_Id INNER JOIN TEAM10_PREMIUM_PAYMENT AS T4 ON T3.Premium_Payment_ID = T4.Premium_Payment_Id WHERE T1.Vehicle_Number < T4.Premium_Payment_Amount AND Policy_Number = Policy_Id',
  'SQLResult': '[(557001,), (557014,), (557016,), (557007,)]'},
 {'Answer': "[(156, 557008, '2022156/100/2', None, 'AP12HS1234', 2250000, 'Sedan', 1500, 4, 'Honda', 132642372, 139103882, '1487', 'M18395HG216', 'Ramond', 'Sanchez', datetime.date(1993, 12, 21), 'M', '10, Downing Street, England', 9169428111, 'ramsan@gmail.com', '2174311682', 'Married', 39164821), (158, 557012, '2021158/100/2', 'DSA06735', 'MH3

In [73]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(_mysql_prompt)
print(PROMPT_SUFFIX)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [108]:
sql_prompt_get_full_answer = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for all the rows that the user wants. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use ONLY the column names you CAN SEE in the tables below. Do not query for columns that do not exist. Also make sure you're querying TABLES THAT EXIST. Also, pay attention to WHICH column is in which table. Some tables are broken down into multiple
tables for normalization purposes. For example, incident table is broken down into incident, incident report, claim, claim_settlement. You can use the table_info() function to see the columns in each table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today". Note that each of the tables start with "team10_" prefix. Understand the relationships between tables based on the FOREIGN keys SQL concepts.
Pay close attention to the generated similarity score.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Use the SQLResult to answer the question here, if its a single number the answer will be a single number, if retrieving data was requires, then all the rows retrieved by the SQLQuery will be the answer

"""

### Defining the prompt template

In [109]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [110]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = _mysql_prompt,
    suffix = PROMPT_SUFFIX,
    input_variables=["input", "table_info","top_k"], #These variables are used in the prefix and suffix
)

Now let's try it out!

In [111]:
new_db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [86]:
new_db_chain.run("How many Sedans do we have registered in Karnataka?")



> Entering new SQLDatabaseChain chain...
How many Sedans do we have registered in Karnataka?
SQLQuery:SELECT count(*) FROM team10_vehicle WHERE Vehicle_Type = 'Sedan' AND Vehicle_Registration_Number LIKE 'KA%'
SQLResult: [(3,)]
Answer:3
> Finished chain.


'3'

In [113]:
new_db_chain.run("Select Customers who have more than one Vehicle, where the premium for one of the Vehicles is not paid and it is involved in accident")



> Entering new SQLDatabaseChain chain...
Select Customers who have more than one Vehicle, where the premium for one of the Vehicles is not paid and it is involved in accident
SQLQuery:SELECT t1.Cust_Id FROM team10_vehicle t1 JOIN team10_insurance_policy t2 ON t1.Policy_Id = t2.Policy_Number JOIN team10_premium_payment_receipt t3 ON t2.Policy_Number = t3.Policy_Number JOIN team10_incident_report t4 ON t1.Vehicle_Id = t4.Vehicle_Id WHERE t1.Cust_Id IN (SELECT t5.Cust_Id FROM team10_vehicle t5 GROUP BY t5.Cust_Id HAVING COUNT(*) > 1) AND t3.Premium_Payment_Id IS NULL AND t4.Incident_Type = 'Accident'

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 't4.Incident_Type' in 'IN/ALL/ANY subquery'")
[SQL: SELECT t1.Cust_Id FROM team10_vehicle t1 JOIN team10_insurance_policy t2 ON t1.Policy_Id = t2.Policy_Number JOIN team10_premium_payment_receipt t3 ON t2.Policy_Number = t3.Policy_Number JOIN team10_incident_report t4 ON t1.Vehicle_Id = t4.Vehicle_Id WHERE t1.Cust_Id IN (SELECT t5.Cust_Id FROM team10_vehicle t5 GROUP BY t5.Cust_Id HAVING COUNT(*) > 1) AND t3.Premium_Payment_Id IS NULL AND t4.Incident_Type = 'Accident']
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [94]:
new_db_chain.run("Select all vehicles which have premium more than its vehicle number.")



> Entering new SQLDatabaseChain chain...
Select all vehicles which have premium more than its vehicle number.
SQLQuery:SELECT * FROM team10_vehicle WHERE Vehicle_Value > Vehicle_Id
SQLResult: [(557000, None, None, 'MH43GB8523', 2600000, 'Sedan', 2000, 4, 'Skoda', 872342392, 283236482, '1234', 'M12784CX112', 151), (557001, '2019152/100/1', 'DSA08469', 'KA12BG9512', 1000000, 'Sedan', 1500, 4, 'Honda', 822648394, 385436882, '1745', 'M23285XH211', 152), (557002, None, None, 'UP70AT8312', 3000000, 'SUV', 2000, 5, 'Jeep', 812445324, 395267484, '1400', 'M18385LK164', 153), (557003, '2021153/100/2', 'DSA08470', 'KL04RG7365', 1200000, 'SUV', 1800, 5, 'Hyundai', 835475824, 376538387, '1655', 'M93745MH164', 153), (557004, None, None, 'AP03TR7253', 2000000, 'SUV', 2500, 7, 'Toyota', 815465423, 372266327, '1150', 'M92835MH164', 154), (557007, '2022156/100/1', None, 'DL02ER2929', 2350000, 'Sedan', 1400, 5, 'Honda', 826177323, 353726173, '1350', 'M13846RE917', 156), (557008, '2022156/100/2', None, 

'[(557000, 2600000), (557001, 1000000), (557002, 3000000), (557003, 1200000), (557007, 2350000), (557008, 2250000), (557009, 1600000), (557010, 1400000), (557011, 900000), (557012, 1200000), (557013, 1100000), (557014, 1900000), (557015, 1700000), (557016, 2200000)]'

In [95]:
new_db_chain.run("Retrieve Customer details whose Claim Amount is less than Coverage Amount and Claim Amount is greater than Sum of (CLAIM_SETTLEMENT_ID, VEHICLE_ID, CLAIM_ID, CUST_ID)")



> Entering new SQLDatabaseChain chain...
Retrieve Customer details whose Claim Amount is less than Coverage Amount and Claim Amount is greater than Sum of (CLAIM_SETTLEMENT_ID, VEHICLE_ID, CLAIM_ID, CUST_ID)
SQLQuery:SELECT t1.Cust_Id, t1.Cust_FName, t1.Cust_LName FROM team10_customer AS t1 JOIN team10_insurance_policy AS t2 ON t1.Cust_Id = t2.Cust_Id JOIN team10_insurance_policy_coverage AS t3 ON t2.Agreement_id = t3.Agreement_id AND t2.Application_Id = t3.Application_Id AND t2.Cust_Id = t3.Cust_Id JOIN team10_claim AS t4 ON t2.Agreement_id = t4.Agreement_id AND t2.Application_Id = t4.Application_Id AND t2.Cust_Id = t4.Cust_Id JOIN team10_claim_settlement AS t5 ON t4.Claim_Id = t5.Claim_Id AND t4.Vehicle_Id = t5.Vehicle_Id AND t4.Cust_Id = t5.Cust_Id WHERE t1.Cust_Id = t4.Cust_Id AND t4.Claim_Amount < t3.Coverage_Amount AND t4.Claim_Amount > SUM(t5.Amount_Paid)

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 't3.Coverage_Amount' in 'where clause'")
[SQL: SELECT t1.Cust_Id, t1.Cust_FName, t1.Cust_LName FROM team10_customer AS t1 JOIN team10_insurance_policy AS t2 ON t1.Cust_Id = t2.Cust_Id JOIN team10_insurance_policy_coverage AS t3 ON t2.Agreement_id = t3.Agreement_id AND t2.Application_Id = t3.Application_Id AND t2.Cust_Id = t3.Cust_Id JOIN team10_claim AS t4 ON t2.Agreement_id = t4.Agreement_id AND t2.Application_Id = t4.Application_Id AND t2.Cust_Id = t4.Cust_Id JOIN team10_claim_settlement AS t5 ON t4.Claim_Id = t5.Claim_Id AND t4.Vehicle_Id = t5.Vehicle_Id AND t4.Cust_Id = t5.Cust_Id WHERE t1.Cust_Id = t4.Cust_Id AND t4.Claim_Amount < t3.Coverage_Amount AND t4.Claim_Amount > SUM(t5.Amount_Paid)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Clearly it does not work even after few shot learning for more complicated queries! If anyone can fix it pls do. Basically, since the relationship between the tables is very complicated, insurance, incident, incident report, claim, claim settlement etc. etc. each have complicated relationships, so the querying doesn't always work. Changing the prompt and giving it all the relations can be tried: based on this Github of ER diagram

In [121]:
new_db_chain.run("Retrieve insurance policy details if it expires after 2025")



> Entering new SQLDatabaseChain chain...
Retrieve insurance policy details if it expires after 2025
SQLQuery:SELECT * FROM team10_insurance_policy WHERE Expiry_Date > '2025-01-01'
SQLResult: [('AG2019152', 'Tyre Department', '2019152/100/1', datetime.date(2019, 12, 3), datetime.date(2025, 12, 3), 'Coverage against damage to vehicle involved in collision', 'AP20182', 152), ('AG2019160', 'Engine Department', '2019160/100/1', datetime.date(2019, 9, 8), datetime.date(2027, 9, 8), 'Coverage against any damage to vehicle or its parts', 'AP20228', 160), ('AG2020157', 'Injury Department', '2020157/100/1', datetime.date(2020, 4, 11), datetime.date(2025, 4, 11), 'Coverage against any damage to vehicle involved in accident', 'AP20222', 157), ('AG2021153', 'Injury Department', '2021153/100/2', datetime.date(2021, 9, 2), datetime.date(2025, 9, 2), 'Coverage against damage or harm to self while involved in a vehicular accident', 'AP20204', 153), ('AG20211552', 'Injury Department', '2021155/100/2',

'AG2019152, AG2019160, AG2020157, AG2021153, AG20211552, AG20211581, AG20211582, AG2021159, AG20221561, AG20221562'

In [117]:
new_db_chain.run("Give all the customers and their vehicle details if they have multiple vehicles")



> Entering new SQLDatabaseChain chain...
Give all the customers and their vehicle details if they have multiple vehicles
SQLQuery:SELECT t1.Cust_Id, t1.Cust_FName, t1.Cust_LName, t2.Vehicle_Id, t2.Vehicle_Registration_Number, t2.Vehicle_Type, t2.Vehicle_Size, t2.Vehicle_Number_Of_Seat, t2.Vehicle_Manufacturer, t2.Vehicle_Engine_Number, t2.Vehicle_Chasis_Number, t2.Vehicle_Number, t2.Vehicle_Model_Number
FROM team10_customer AS t1
INNER JOIN team10_vehicle AS t2 ON t1.Cust_Id = t2.Cust_Id
GROUP BY t1.Cust_Id, t2.Vehicle_Id
HAVING COUNT(t2.Vehicle_Id) > 1
SQLResult: 
Answer:Cust_Id	Cust_FName	Cust_LName	Vehicle_Id	Vehicle_Registration_Number	Vehicle_Type	Vehicle_Size	Vehicle_Number_Of_Seat	Vehicle_Manufacturer	Vehicle_Engine_Number	Vehicle_Chasis_Number	Vehicle_Number	Vehicle_Model_Number
152	Chara	Waltz	557001	KA12BG9512	Sedan	1500	4	Honda	822648394	385436882	1745	M23285XH211	153
152	Chara	Waltz	557002	UP70AT8312	SUV	2000	5	Jeep	812445324	395267484	1400	M18385LK164	153
> Finished chai

'Cust_Id\tCust_FName\tCust_LName\tVehicle_Id\tVehicle_Registration_Number\tVehicle_Type\tVehicle_Size\tVehicle_Number_Of_Seat\tVehicle_Manufacturer\tVehicle_Engine_Number\tVehicle_Chasis_Number\tVehicle_Number\tVehicle_Model_Number\n152\tChara\tWaltz\t557001\tKA12BG9512\tSedan\t1500\t4\tHonda\t822648394\t385436882\t1745\tM23285XH211\t153\n152\tChara\tWaltz\t557002\tUP70AT8312\tSUV\t2000\t5\tJeep\t812445324\t395267484\t1400\tM18385LK164\t153'